In [ ]:
from langchain.document_loaders import PyMuPDFLoader

/Users/akashdeepsangwan/Desktop/Code/GenAI/RAG/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Load the document - Template of PyMuP

docs = PyMuPDFLoader("/Users/akashdeepsangwan/Desktop/Code/GenAI/RAG/Can AI Build Systems (1).pdf").load()
type(docs)

list

#### PDF Extracted provide an array, Each element is a whole page


In [82]:
print((docs[2].page_content))

our methodology aims to provide more structured, end-to- end
support for architects, ranging from early analysis to design-
level detailing.
III. METHODOLOGY
We applied various prompt engineering techniques to eval-
uate the performance of different Large Language Models
(LLMs) in generating architecture of software systems given
their textual descriptions in natural langauge. Specifically,
we benchmarked three prominent LLMs under each of these
prompting strategies: OpenAI o3, the latest reasoning-tuned
successor to the GPT-4 family; Gemini 2.0 Gemini, Google’s
latency-optimised variant of Gemini designed for rapid iter-
ative calls; and Llama 4 Llama, Meta’s fourth-generation
open-weight model fine-tuned for software-engineering tasks.
These models are selected based on popularity and their
wide applicability on software development tasks. We evaluate
two complementary workflows for turning a natural-language
system description into a PlantUML component diagram. We
decided to use UML

In [37]:
# Create chunks of this PDF
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap = 65,
    length_function = len
)

In [38]:
chunks = splitter.split_documents(docs)
print(f"No of chunks {len(chunks)}")


No of chunks 101


In [39]:
print(type(chunks[1].page_content))

<class 'str'>


In [40]:
print(chunks[1].page_content)


tion cohesively. Despite being introduced only recently, Large
Language Models (LLMs) have shown great potential in au-
tomating low-level software engineering tasks, such as coding
and test generation. However, the application of LLMs on high
level cognitive tasks, such as architecture and design, has not
been explored as much. To that end, this study evaluates the
capability of LLMs—specifically Llama 4 Llama, Gemini 2.0
Gemini, and OpenAI o3 in generating software architecture
given unstructured textual requirements in natural language.


In [41]:
for i in range(5) :
    print(f"Chunk {i+1}")
    print(f"{chunks[i].page_content[:100] + "..." + chunks[i].page_content[len(chunks[i].page_content)-200:] }")
    print("-------------------------------------------------------------------------")

    

Chunk 1
Can AI Build Systems? An Exploratory Study on
Generating Software Architecture with LLMs
Marios Foka...in shaping
the quality, maintainability, and integrity of a software system,
providing a blueprint that ensures that all components func-
tion cohesively. Despite being introduced only recently, Large
-------------------------------------------------------------------------
Chunk 2
tion cohesively. Despite being introduced only recently, Large
Language Models (LLMs) have shown gre...his study evaluates the
capability of LLMs—specifically Llama 4 Llama, Gemini 2.0
Gemini, and OpenAI o3 in generating software architecture
given unstructured textual requirements in natural language.
-------------------------------------------------------------------------
Chunk 3
given unstructured textual requirements in natural language.
We employ prompt engineering techniques..., akin to a
human editing a preliminary draft for improvements. Our results
indicate that state of the art models lik

### Has Good Amount of OverLap - Task Done

- Next : Will apply semantic chunking in Chunking process

In [43]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

Embeddings= HuggingFaceEmbeddings(
    model = "sentence-transformers/all-MiniLM-L6-v2"
)


In [55]:
sentences = [chunk.page_content for chunk in chunks]
print(len(sentences)== len(chunks))

True


In [89]:
print(type(sentences))

<class 'list'>


In [58]:
# Convert the text into embeddings now
doc_embeddings = Embeddings.embed_documents(sentences)
print(len(doc_embeddings))
print(len(sentences))

101
101


Cosine similarity max is 1, when the angle b/w two vector is zero, means they overlap each other 

In [ ]:
threshold = 0.7  # control chunk tightness
new_chunks = []
current_chunk=[sentences[0]]

## Step 4: Semantic grouping based on threshold

for i in range(1, len(sentences)):
    sim = cosine_similarity(
        [doc_embeddings[i - 1]],
        [doc_embeddings[i]]
    )[0][0]

    if sim>=threshold:
        current_chunk.append(sentences[i])
    else:
        new_chunks.append(" ".join(current_chunk))
        current_chunk=[sentences[i]]

# Append the last chunk
new_chunks.append(" ".join(current_chunk))

# Output the chunks
print(" Semantic Chunks:")
for idx, chunk in enumerate(new_chunks):
    print(f"\nChunk {idx+1}:\n{new_chunks}")


In [79]:
for i in range(4,10) :
    print(f"New Chunk {i+1}")
    print(new_chunks[i][:100])
    print("******************************************")
    print(f"Older Chunk {i+1}")
    print(chunks[i].page_content[:100])

    print("-----------------------------------------")

New Chunk 5
Index Terms—software architecture, large language models,
UML, software design, software requirement
******************************************
Older Chunk 5
Index Terms—software architecture, large language models,
UML, software design, software requirement
-----------------------------------------
New Chunk 6
These
methodologies enable systematic development while main-
taining intellectual control over syst
******************************************
Older Chunk 6
structures, data access protocols, scalability, and performance
optimization [1]. Architectural deci
-----------------------------------------
New Chunk 7
software architects in their architectural design decisions and
evaluation to reduce the time requir
******************************************
Older Chunk 7
These
methodologies enable systematic development while main-
taining intellectual control over syst
-----------------------------------------
New Chunk 8
even writing and debugging software code. The app

#### After Semantic chunking, the chunks got combined that's why the length of new_chunks is less than older chunks

In [80]:
print(len(new_chunks))
print(len(chunks))

81
101


#### Next : Create the new embeddings of new_chunks and store them in the vector store

In [88]:
print(type(new_chunks))

<class 'list'>


In [ ]:
# this step can be skipped when using FAISS
new_embeddings= Embeddings.embed_documents(new_chunks) # take array of strings as a input


81


### Store the embeddings into a vector store

In [ ]:
from langchain_core.documents import Document

semantic_document=[
    Document(
        page_content = chunk_text,
        metadata = {
            'chunk_type' : 'semantic',
            'source' : "Can AI Build Systems (1).pdf"
        }
    )

    for chunk_text in new_chunks
]

In [97]:
from langchain.vectorstores import FAISS


vector_store = FAISS.from_documents(
    documents = semantic_document,
    embedding = Embeddings
)

In [98]:
vector_store